# How does Julia look like Matlab but run like C?

  * Just-in-time compilation (JIT)
      * user-level code is compiled to machine code on-the-fly
      
      
      
  * Meticulous type system designed to maximize impact of JIT
      * type inference
      * type stability
      * multiple dispatch



## Just-in-time compilation 

Define a simple function and run it twice

In [7]:
f(x) = x^3 - 2

f (generic function with 1 method)

In [8]:
@time f(0.3);
@time f(0.3);

  0.005786 seconds (1.19 k allocations: 74.612 KiB)
  0.000002 seconds (5 allocations: 176 bytes)


The second evaluation is thousands of times faster than the first! Why? 

The first run includes a compilation of user code to machine code.

The second run just executes the machine code. 

### Compilation to machine code, in stages

In [5]:
@code_lowered f(7.0)   # show f(x) in Julia's abstract syntax tree

CodeInfo(:(begin 
        nothing
        return (Base.literal_pow)(Main.^, x, (Core.apply_type)(Base.Val, 3)) - 2
    end))

In [6]:
@code_typed f(7.0)     # show f(x) in abstract syntax tree with types determined 

CodeInfo(:(begin 
        return (Base.sub_float)((Base.mul_float)((Base.mul_float)(x, x)::Float64, x)::Float64, (Base.sitofp)(Float64, 2)::Float64)::Float64
    end))=>Float64

In [7]:
@code_llvm f(7.0)       # show f(x) in LLVM (compiler) intermediate language


define double @julia_f_60969(double) #0 !dbg !5 {
top:
  %1 = fmul double %0, %0
  %2 = fmul double %1, %0
  %3 = fadd double %2, -2.000000e+00
  ret double %3
}


In [8]:
@code_native f(7.0)     # show f(x) in Intel IA-64 assembly language

	.text
Filename: In[1]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 1
	movapd	%xmm0, %xmm1
	mulsd	%xmm1, %xmm1
	mulsd	%xmm0, %xmm1
	movabsq	$140452003107272, %rax  # imm = 0x7FBD87C0D1C8
	addsd	(%rax), %xmm1
	movapd	%xmm1, %xmm0
	popq	%rbp
	retq
	nopw	%cs:(%rax,%rax)


## Type inference

Julia figures out the types of untyped variables --crucial for compiling to machine code!

In [9]:
@code_llvm f(7.0)   # f applied to a Float64 (C's "double" type)


define double @julia_f_60969(double) #0 !dbg !5 {
top:
  %1 = fmul double %0, %0
  %2 = fmul double %1, %0
  %3 = fadd double %2, -2.000000e+00
  ret double %3
}


In [9]:
@code_llvm f(7f0)   # f applied to a Float32  (C's "float" type)


define float @julia_f_61165(float) #0 !dbg !5 {
top:
  %1 = fmul float %0, %0
  %2 = fmul float %1, %0
  %3 = fadd float %2, -2.000000e+00
  ret float %3
}


In [10]:
@code_llvm f(7)     # f applied to an Int64   (C's "int" type)


define i64 @julia_f_61125(i64) #0 !dbg !5 {
top:
  %1 = mul i64 %0, %0
  %2 = mul i64 %1, %0
  %3 = add i64 %2, -2
  ret i64 %3
}


In [5]:
@code_llvm f(7//1) # f applied to a Rational, much more complex


define void @julia_f_61027(%Rational* noalias nocapture sret, %Rational* nocapture readonly dereferenceable(16)) #0 !dbg !5 {
top:
  %2 = alloca %Rational, align 8
  %3 = alloca %Rational, align 8
  %4 = alloca %Rational, align 8
  %5 = alloca %Rational, align 8
  %6 = alloca %Rational, align 8
  call void @"julia_*_61025"(%Rational* noalias nocapture nonnull sret %3, %Rational* nocapture nonnull readonly %1, %Rational* nocapture nonnull readonly %1)
  call void @"julia_*_61025"(%Rational* noalias nocapture nonnull sret %2, %Rational* nocapture nonnull readonly %3, %Rational* nocapture nonnull readonly %1)
  %7 = getelementptr inbounds %Rational, %Rational* %2, i64 0, i32 0
  %8 = load i64, i64* %7, align 8
  %9 = getelementptr inbounds %Rational, %Rational* %2, i64 0, i32 1
  %10 = load i64, i64* %9, align 8
  call void @julia_Type_61013(%Rational* noalias nocapture nonnull sret %5, i8** inttoptr (i64 140039431844048 to i8**), i64 %8, i64 %10)
  call void @julia_Type_61013(%Rational*

## Type stability

Just-in-time compilation produces most efficient machine code when types of temporaries and return values can be determined at compile-time. 

In Julia, `sqrt` is real-to-real, and `sqrt(-1)` is an error! Matlab avoids this problem by 
making all numbers complex, and thus doubling the cost and size of all real-valued mathematics --in Matlab, a real-valued scalar is a 1 x 1 complex matrix!

In [13]:
sqrt(1.0)      

1.0

In [14]:
sqrt(-1.0)

LoadError: DomainError:
sqrt will only return a complex result if called with a complex argument. Try sqrt(complex(x)).

In [27]:
sqrt(-1.0 + 0.0im)   # In Julia, if you want a complex sqrt result, use the complex sqrt function

0.0 + 1.0im

## Multiple dispatch

Most functions have multiple versions optimized for their particular input types. Selection is done at compile time
if types can be inferred, run time if they can't.

In [1]:
methods(^)

# 52 methods for generic function "^":
^(x::Bool, y::BigInt) in Base.GMP at gmp.jl:484
^(x::Bool, y::Bool) in Base at bool.jl:92
^(a::Float16, b::Float16) in Base at float.jl:372
^(x::Float16, y::Integer) in Base.Math at math.jl:703
^(x::Float64, y::Float64) in Base.Math at math.jl:699
^(x::Float64, y::Integer) in Base.Math at math.jl:701
^(x::Float32, y::Float32) in Base.Math at math.jl:700
^(x::Float32, y::Integer) in Base.Math at math.jl:702
^(x::BigInt, y::Bool) in Base.GMP at gmp.jl:481
^(x::BigInt, y::BigInt) in Base.GMP at gmp.jl:480
^(x::BigInt, y::UInt64) in Base.GMP at gmp.jl:455
^(x::BigInt, y::Integer) in Base.GMP at gmp.jl:482
^(x::BigFloat, y::BigInt) in Base.MPFR at mpfr.jl:507
^(x::BigFloat, y::BigFloat) in Base.MPFR at mpfr.jl:489
^(x::BigFloat, y::Union{UInt16, UInt32, UInt64, UInt8}) in Base.MPFR at mpfr.jl:495
^(x::BigFloat, y::Union{Int16, Int32, Int64, Int8}) in Base.MPFR at mpfr.jl:501
^(x::BigFloat, y::Unsigned) in Base.MPFR at mpfr.jl:513
^(x::BigFloat, y::Integer) in Base.MPFR at mpfr.jl:512
^(::Irrational{:e}, x::Irrational) in Base at irrationals.jl:219
^(::Irrational{:e}, x::Rational) in Base at irrationals.jl:219
^(::Irrational{:e}, x::Integer) in Base at irrationals.jl:219
^(::Irrational{:e}, x::Number) in Base at irrationals.jl:219
^(x::Integer, y::Bool) in Base at bool.jl:93
^(z::Complex{T}, p::Complex{T}) where T<:AbstractFloat in Base at complex.jl:571
^(z::T, p::T) where T<:Complex in Base at complex.jl:620
^(z::Complex{#s45} where #s45<:AbstractFloat, n::Bool) in Base at complex.jl:680
^(z::Complex{#s45} where #s45<:Integer, n::Bool) in Base at complex.jl:681
^(z::Complex{#s45} where #s45<:Rational, n::Bool) in Base at rational.jl:427
^(z::Complex, n::Bool) in Base at complex.jl:677
^(z::Complex{#s45} where #s45<:AbstractFloat, n::Integer) in Base at complex.jl:683
^(z::Complex{#s45} where #s45<:Integer, n::Integer) in Base at complex.jl:685
^(z::Complex{#s45} where #s45<:Rational, n::Integer) in Base at rational.jl:429
^(z::Complex, n::Integer) in Base at complex.jl:678
^(x::Rational, n::Integer) in Base at rational.jl:420
^(x::T, y::Rational) where T<:AbstractFloat in Base at rational.jl:424
^(x::Complex{T}, y::Rational) where T<:AbstractFloat in Base at rational.jl:425
^(x::Number, y::Rational) in Base at rational.jl:423
^(x::Integer, y::BigInt) in Base.GMP at gmp.jl:483
^(x::T, p::T) where T<:Integer in Base at intfuncs.jl:194
^(x::Number, p::Integer) in Base at intfuncs.jl:195
^(x::Irrational, y::Irrational) in Base at irrationals.jl:109
^(x::T, y::T) where T<:Number in Base at promotion.jl:339
^(x::Number, y::Number) in Base at promotion.jl:281
^(s::AbstractString, r::Integer) in Base at strings/types.jl:154
^(A::Symmetric{T,S} where S<:(AbstractArray{T,2} where T), p::Integer) where T<:Real in Base.LinAlg at linalg/symmetric.jl:461
^(A::Hermitian, p::Integer) in Base.LinAlg at linalg/symmetric.jl:477
^(A::AbstractArray{T,2}, p::Integer) where T in Base.LinAlg at linalg/dense.jl:332
^(A::Symmetric{T,S} where S<:(AbstractArray{T,2} where T), p::Real) where T<:Real in Base.LinAlg at linalg/symmetric.jl:468
^(A::Hermitian{T,S} where S<:(AbstractArray{T,2} where T), p::Real) where T in Base.LinAlg at linalg/symmetric.jl:489
^(A::AbstractArray{T,2}, p::Real) where T in Base.LinAlg at linalg/dense.jl:335
^(A::AbstractArray{T,2} where T, p::Number) in Base.LinAlg at linalg/dense.jl:391
^(x, p::Integer) in Base at intfuncs.jl:196

 Type inference and multiple dispatch allow the just-in-time compiler to translate complex chains of type-stable Julia code into efficient machine code.

## Comparison: iterated logistic map in Julia, C++, and Matlab

Define $f(x) = 4x(1-x)$, generate millionth iterate function $f^N(x)$

In [1]:
# define function that, given an f, returns iterated function f^N
function iterator(f, N)
    
    # construct f^N
    function fN(x)
      for i ∈ 1:N             
        x = f(x)
      end
      x
    end    
    
    fN     # return f^N
end

# define logistic map function
f(x) = 4*x*(1-x)

# use iterator function to constuct millionth iterate of logistic map
fᴺ  = iterator(f, 10^6)  

(::fN) (generic function with 1 method)

In [4]:
@time fᴺ(0.67)
@time fᴺ(0.67)

  0.003977 seconds (5 allocations: 176 bytes)
  0.003629 seconds (5 allocations: 176 bytes)


0.10116885334547539

### Equivalent C++ code

note: starting semicolon tells Julia to execute Unix shell code

In [16]:
; pwd

/home/gibson/gitworking/whyjulia


In [18]:
; cat fmillion.cpp

#include <stdlib.h>
#include <iostream>
#include <iomanip>
#include <ctime>

using namespace std;

double f(double x) {
  return 4*x*(1-x);
}

int main(int argc, char* argv[]) {
  double x = argc > 1 ? atof(argv[1]) : 0.0;

  double t0 = clock();
  for (int n=0; n<1000000; ++n)
    x = f(x);
  double t1 = clock();

  cout << "t = " << (t1-t0)/CLOCKS_PER_SEC << " seconds" << endl;
  cout << setprecision(17);
  cout << "x = " << x << endl;
  
  return 0;
}
  


In [19]:
; g++ -O3 -o fmillion fmillion.cpp

### Execution time for C++

In [5]:
; fmillion 0.67

t = 0.003234 seconds
x = 0.10116885334547539


### Execution time for Julia

In [7]:
print("t="); 
@time x = fᴺ(0.67);
@show x;

t=  0.003590 seconds (5 allocations: 176 bytes)
x = 0.10116885334547539


Execution times $t$ are comparable. Sometimes Julia is faster, sometimes C. Millionth iterates $x$ are the same, indicate same sequence of floating-point operations. 

### Execution time in Matlab

```
>> tic(); x=fN(0.67); t=toc();
>> t,x
t = 0.048889000000000
x = 0.101168853345475
```
Same number, but about five to ten times slower than Julia or C++.